# Predicción de Retención de Clientes para Beta Bank

## Introducción
Los clientes de Beta Bank están abandonando el banco gradualmente cada mes. Dado que es más rentable retener a los clientes existentes que atraer nuevos, Beta Bank necesita un modelo predictivo que pueda identificar qué clientes probablemente abandonarán el banco en el futuro cercano. Este proyecto tiene como objetivo desarrollar un modelo con el máximo valor F1 posible para predecir la probabilidad de churn de los clientes. Además del F1, evaluaremos la métrica AUC-ROC para una comparación integral.

### Descripción de los Datos
El conjunto de datos proporcionado contiene las siguientes características:

RowNumber: Índice de cadena de datos

CustomerId: Identificador único del cliente

Surname: Apellido del cliente

CreditScore: Puntuación crediticia del cliente

Geography: País de residencia del cliente

Gender: Género del cliente

Age: Edad del cliente

Tenure: Período de madurez del depósito a plazo fijo del cliente (años)

Balance: Saldo de la cuenta del cliente

NumOfProducts: Número de productos bancarios utilizados por el cliente

HasCrCard: Indicador de si el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)

IsActiveMember: Actividad del cliente (1 - sí; 0 - no)

EstimatedSalary: Salario estimado del cliente

El objetivo es predecir la variable Exited, que indica si el cliente ha abandonado el banco (1 - sí; 0 - no).

## Importación de Librerías Necesarias

In [2]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from sklearn.impute import SimpleImputer

## Descargar y Preparar los Datos

In [3]:
# Cargar los datos
data_path = '/datasets/Churn.csv'
df = pd.read_csv(data_path)

# Mostrar las primeras filas del DataFrame
print(df.head())

# Información general sobre el DataFrame
print(df.info())

# Estadísticas descriptivas de las variables
print(df.describe())

# Verificar valores nulos
print(df.isnull().sum())


   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

### Conclusión parcial

hemos descargado y examinado el conjunto de datos proporcionado por Beta Bank. Los datos contienen 10,000 registros y 14 columnas, donde cada columna representa una característica del cliente, como 'CreditScore', 'Geography', 'Gender', 'Age', entre otras, y la columna objetivo 'Exited' indica si el cliente ha abandonado el banco o no.

En la exploración inicial, observamos que:

La mayoría de las columnas no contienen valores nulos, con la excepción de la columna 'Tenure' que presenta 909 valores nulos.

Las variables categóricas incluyen 'Geography' y 'Gender'.

El conjunto de datos está bien estructurado y contiene una mezcla de variables numéricas y categóricas.

Estos hallazgos nos permitirán planificar el preprocesamiento necesario, como el manejo de valores nulos y la codificación de variables categóricas, antes de entrenar los modelos de machine learning.

## Preprocesamiento de Datos
En este paso, realizaremos el preprocesamiento necesario para preparar los datos para el entrenamiento del modelo. Esto incluye manejar los valores nulos, codificar las variables categóricas, y dividir los datos en conjuntos de entrenamiento y prueba. El preprocesamiento es crucial para garantizar que los datos estén en un formato adecuado para los algoritmos de machine learning y para mejorar la calidad de nuestros modelos.

### Columna 'Tenure'

En nuestro análisis inicial, notamos que la columna 'Tenure' contiene valores nulos. Decidiremos la mejor estrategia para manejarlos, como la imputación con la mediana o la eliminación de las filas con valores nulos.

In [4]:
# Verificar valores nulos
print(df.isnull().sum())

# Imputar valores nulos en 'Tenure' con la mediana
imputer = SimpleImputer(strategy='median')
df['Tenure'] = imputer.fit_transform(df[['Tenure']])

# Verificar valores nulos después de la imputación
print(df.isnull().sum())

# Mostrar las primeras filas para verificar el resultado de la imputación
print(df.head())

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64
RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Fem

### Codificación de Variables Categóricas
En este paso, convertiremos las variables categóricas ('Geography' y 'Gender') en variables numéricas utilizando One-Hot Encoding (OHE). Esto nos permitirá utilizar estas variables en nuestros modelos de aprendizaje automático.

In [5]:
# Aplicar One-Hot Encoding a las columnas 'Geography' y 'Gender'
df = pd.get_dummies(df, columns=['Geography', 'Gender'], drop_first=True)

# Verificar las primeras filas después de la codificación
print(df.head())

   RowNumber  CustomerId   Surname  CreditScore  Age  Tenure    Balance  \
0          1    15634602  Hargrave          619   42     2.0       0.00   
1          2    15647311      Hill          608   41     1.0   83807.86   
2          3    15619304      Onio          502   42     8.0  159660.80   
3          4    15701354      Boni          699   39     1.0       0.00   
4          5    15737888  Mitchell          850   43     2.0  125510.82   

   NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  Exited  \
0              1          1               1        101348.88       1   
1              1          0               1        112542.58       0   
2              3          1               0        113931.57       1   
3              2          0               0         93826.63       0   
4              1          1               1         79084.10       0   

   Geography_Germany  Geography_Spain  Gender_Male  
0                  0                0            0  
1         

#### Conclusión parcial

Este paso ha transformado efectivamente nuestras variables categóricas, permitiendo una mejor comprensión y procesamiento de los datos por los algoritmos de aprendizaje automático. Ahora podemos proceder a la siguiente etapa del preprocesamiento de datos, asegurándonos de que los datos estén listos para la modelización.

### División de Datos en Conjuntos de Entrenamiento, Validación y Prueba
En este paso, dividiremos los datos en tres conjuntos: entrenamiento, validación y prueba. Esta división nos permitirá entrenar el modelo en un conjunto de datos, validar su rendimiento y, finalmente, evaluar su rendimiento en un conjunto de prueba independiente.

In [6]:
# Definir características y objetivo
features = df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited'])
target = df['Exited']

# Dividir los datos en conjunto de entrenamiento (60%) y conjunto restante (40%)
features_train, features_temp, target_train, target_temp = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target
)

# Dividir el conjunto restante en validación (50%) y prueba (50%) para obtener 20% cada uno
features_valid, features_test, target_valid, target_test = train_test_split(
    features_temp, target_temp, test_size=0.5, random_state=12345, stratify=target_temp
)

# Mostrar los tamaños de los conjuntos
print(f'Tamaño del conjunto de entrenamiento: {features_train.shape[0]}')
print(f'Tamaño del conjunto de validación: {features_valid.shape[0]}')
print(f'Tamaño del conjunto de prueba: {features_test.shape[0]}')

Tamaño del conjunto de entrenamiento: 6000
Tamaño del conjunto de validación: 2000
Tamaño del conjunto de prueba: 2000


#### Conclusión parcial 

Hemos dividido con éxito los datos en conjuntos de entrenamiento, validación y prueba, manteniendo la proporción de clases en cada conjunto gracias a la estratificación. Esta división nos permitirá entrenar, validar y probar nuestros modelos de manera efectiva, asegurando que los resultados sean representativos y robustos.

## Entrenar Modelos Iniciales sin Considerar el Desequilibrio de Clases
En este paso, entrenaremos varios modelos de clasificación sin tener en cuenta el desequilibrio de clases en los datos. Evaluaremos su rendimiento en el conjunto de validación y compararemos los resultados.

In [7]:
# Entrenar y evaluar modelo Decision Tree
dt_model = DecisionTreeClassifier(random_state=12345)
dt_model.fit(features_train, target_train)
dt_predictions = dt_model.predict(features_valid)
dt_f1 = f1_score(target_valid, dt_predictions)
dt_auc_roc = roc_auc_score(target_valid, dt_model.predict_proba(features_valid)[:, 1])

# Entrenar y evaluar modelo Random Forest
rf_model = RandomForestClassifier(random_state=12345)
rf_model.fit(features_train, target_train)
rf_predictions = rf_model.predict(features_valid)
rf_f1 = f1_score(target_valid, rf_predictions)
rf_auc_roc = roc_auc_score(target_valid, rf_model.predict_proba(features_valid)[:, 1])

# Entrenar y evaluar modelo Logistic Regression
lr_model = LogisticRegression(random_state=12345, max_iter=1000)
lr_model.fit(features_train, target_train)
lr_predictions = lr_model.predict(features_valid)
lr_f1 = f1_score(target_valid, lr_predictions)
lr_auc_roc = roc_auc_score(target_valid, lr_model.predict_proba(features_valid)[:, 1])

# Mostrar las métricas de rendimiento
print(f'Decision Tree - F1: {dt_f1:.4f}, AUC-ROC: {dt_auc_roc:.4f}')
print(f'Random Forest - F1: {rf_f1:.4f}, AUC-ROC: {rf_auc_roc:.4f}')
print(f'Logistic Regression - F1: {lr_f1:.4f}, AUC-ROC: {lr_auc_roc:.4f}')

Decision Tree - F1: 0.5048, AUC-ROC: 0.6904
Random Forest - F1: 0.6106, AUC-ROC: 0.8622
Logistic Regression - F1: 0.0619, AUC-ROC: 0.6995


### Conlusión parcial
Dado que Random Forest ha mostrado el mejor rendimiento inicial, procederemos a mejorar la calidad del modelo abordando el desequilibrio de clases en los datos. Utilizaremos al menos dos técnicas para corregir el desequilibrio y evaluaremos su impacto en el rendimiento del modelo.

## Mejorar la Calidad del Modelo
En este paso, mejoraremos la calidad del modelo abordando el desequilibrio de clases. Utilizaremos dos enfoques principales para corregir el desequilibrio: sobremuestreo (oversampling) y submuestreo (undersampling). Evaluaremos el impacto de cada técnica en el rendimiento del modelo utilizando Random Forest, que mostró el mejor rendimiento en los pasos anteriores.

In [17]:
# Separar clases mayoritarias y minoritarias
df_majority = df[df.Exited == 0]
df_minority = df[df.Exited == 1]

# Aumentar la clase minoritaria
df_minority_oversampled = resample(df_minority, 
                                   replace=True,     
                                   n_samples=len(df_majority),    
                                   random_state=123) 

# Combinar las clases mayoritarias y minoritarias aumentadas
df_oversampled = pd.concat([df_majority, df_minority_oversampled])

# Verificar el nuevo balance de clases
print(df_oversampled.Exited.value_counts())

# Dividir los datos aumentados
X = df_oversampled.drop(['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1)
y = df_oversampled['Exited']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=12345)

# Entrenar y evaluar el modelo Random Forest con sobremuestreo
model_oversampling = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=8)
model_oversampling.fit(X_train, y_train)
predictions = model_oversampling.predict(X_valid)

f1_oversampling = f1_score(y_valid, predictions)
auc_roc_oversampling = roc_auc_score(y_valid, model_oversampling.predict_proba(X_valid)[:, 1])

print(f'Oversampling - F1: {f1_oversampling:.4f}, AUC-ROC: {auc_roc_oversampling:.4f}')


0    7963
1    7963
Name: Exited, dtype: int64
Oversampling - F1: 0.8192, AUC-ROC: 0.9030


In [18]:
# Reducir la clase mayoritaria
df_majority_downsampled = resample(df_majority, 
                                   replace=False,   
                                   n_samples=len(df_minority),     
                                   random_state=123) 

# Combinar clases mayoritarias y minoritarias reducidas
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Verificar el nuevo balance de clases
print(df_downsampled.Exited.value_counts())

# Dividir los datos reducidos
X = df_downsampled.drop(['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1)
y = df_downsampled['Exited']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=12345)

# Entrenar y evaluar el modelo Random Forest con submuestreo
model_undersampling = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=8)
model_undersampling.fit(X_train, y_train)
predictions = model_undersampling.predict(X_valid)

f1_undersampling = f1_score(y_valid, predictions)
auc_roc_undersampling = roc_auc_score(y_valid, model_undersampling.predict_proba(X_valid)[:, 1])

print(f'Undersampling - F1: {f1_undersampling:.4f}, AUC-ROC: {auc_roc_undersampling:.4f}')


0    2037
1    2037
Name: Exited, dtype: int64
Undersampling - F1: 0.7745, AUC-ROC: 0.8608


### Conclusión parcial

**Sobremuestreo (Oversampling)**:
- **Balance de Clases**: El sobremuestreo ha equilibrado perfectamente las clases, resultando en 7963 observaciones para cada clase (0 y 1).
- **Rendimiento del Modelo**: El modelo Random Forest entrenado con el conjunto de datos sobremuestreado ha alcanzado un F1-score de 0.8192 y un AUC-ROC de 0.9030. Estos valores indican un buen equilibrio entre precisión y recall, y una alta capacidad para distinguir entre clases positivas y negativas.

**Submuestreo (Undersampling)**:
- **Balance de Clases**: El submuestreo ha equilibrado las clases a un nivel más reducido, con 2037 observaciones para cada clase (0 y 1).
- **Rendimiento del Modelo**: El modelo Random Forest entrenado con el conjunto de datos submuestreado ha alcanzado un F1-score de 0.7745 y un AUC-ROC de 0.8608. Aunque estos valores son buenos, son ligeramente inferiores a los obtenidos con el sobremuestreo.

**Comparación y Observaciones**:
- **F1-Score**: El modelo con sobremuestreo tiene un F1-score más alto (0.8192) en comparación con el modelo con submuestreo (0.7745). Esto sugiere que el modelo con sobremuestreo maneja mejor el equilibrio entre precisión y recall.
- **AUC-ROC**: El AUC-ROC del modelo con sobremuestreo (0.9030) es también superior al del modelo con submuestreo (0.8608), indicando una mejor capacidad de discriminación entre las clases positivas y negativas.

**Observación**:
El modelo entrenado con sobremuestreo muestra un mejor rendimiento en términos de F1-score y AUC-ROC en comparación con el modelo entrenado con submuestreo. Por lo tanto, el enfoque de sobremuestreo es más efectivo para abordar el desequilibrio de clases en este conjunto de datos. En el siguiente paso, utilizaremos este modelo mejorado para la prueba final y evaluaremos su rendimiento en el conjunto de prueba.


## Prueba Final del Modelo

En este paso, entrenaremos el modelo Random Forest utilizando el enfoque de sobremuestreo en todo el conjunto de datos y evaluaremos su rendimiento en el conjunto de prueba. Nuestro objetivo es verificar si el modelo mantiene un buen rendimiento en términos de F1-score y AUC-ROC en datos no vistos anteriormente.

### Entrenamiento Final del Modelo con Sobremuestreo

Entrenaremos el modelo Random Forest con el conjunto de datos completo y equilibrado mediante sobremuestreo. Luego, evaluaremos su rendimiento en el conjunto de prueba y calcularemos las métricas F1-score y AUC-ROC.

In [24]:
# Aumentar la clase minoritaria
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     
                                 n_samples=len(df_majority),    
                                 random_state=123) 

# Combinar las clases mayoritarias y minoritarias aumentadas
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# Verificar el nuevo balance de clases
print(df_upsampled.Exited.value_counts())

# Dividir los datos aumentados en entrenamiento y prueba
X = df_upsampled.drop(['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1)
y = df_upsampled['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345, stratify=y)

# Entrenar el modelo con el conjunto sobremuestreado completo
best_model = RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=8)
best_model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
test_predictions = best_model.predict(X_test)
test_f1 = f1_score(y_test, test_predictions)
test_auc_roc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])

print(f'Final Test - F1: {test_f1:.4f}, AUC-ROC: {test_auc_roc:.4f}')

0    7963
1    7963
Name: Exited, dtype: int64
Final Test - F1: 0.8192, AUC-ROC: 0.9024


## Conclusión General del Proyecto

En este proyecto, desarrollamos un modelo predictivo para identificar clientes que probablemente abandonarán Beta Bank. A continuación, se resumen los pasos principales y hallazgos del proyecto:

### Exploración y Preprocesamiento de Datos
- **Exploración Inicial:** Descargamos y examinamos los datos proporcionados, identificando valores nulos y características categóricas que requirieron tratamiento.
- **Imputación y Codificación:** Imputamos los valores nulos en la columna 'Tenure' utilizando la mediana y codificamos las variables categóricas 'Geography' y 'Gender' utilizando One-Hot Encoding (OHE).

### División de Datos
- **División Estratificada:** Dividimos los datos en conjuntos de entrenamiento (60%), validación (20%) y prueba (20%), manteniendo la proporción de clases en cada conjunto mediante estratificación.

### Entrenamiento Inicial de Modelos
- **Modelos Baseline:** Entrenamos y evaluamos varios modelos de clasificación (Decision Tree, Random Forest, Logistic Regression) sin abordar el desequilibrio de clases. El modelo Random Forest mostró el mejor rendimiento inicial con un F1-score de 0.6106 y un AUC-ROC de 0.8622 en el conjunto de validación.

### Mejora del Modelo
- **Abordaje del Desequilibrio de Clases:** Utilizamos técnicas de sobremuestreo y submuestreo para corregir el desequilibrio de clases. El modelo Random Forest con sobremuestreo ofreció el mejor rendimiento, alcanzando un F1-score de 0.8192 y un AUC-ROC de 0.9030 en el conjunto de validación.

### Prueba Final del Modelo
- **Entrenamiento Completo:** Entrenamos el modelo Random Forest con el conjunto de datos completo y sobremuestreado, y evaluamos su rendimiento en el conjunto de prueba.
- **Resultados:** El modelo final alcanzó un F1-score de 0.8192 y un AUC-ROC de 0.9024 en el conjunto de prueba, demostrando una alta capacidad para predecir correctamente qué clientes probablemente abandonarán el banco.

### Sugerencias para Futuras Decisiones
1. **Optimización Continua:** Continuar optimizando los hiperparámetros del modelo utilizando técnicas avanzadas para mejorar aún más el rendimiento.
2. **Exploración de Nuevos Algoritmos:** Explorar algoritmos de aprendizaje automático más avanzados que pueden ofrecer un mejor rendimiento en problemas de clasificación desequilibrada.
3. **Feature Engineering:** Investigar y crear nuevas características derivadas de las existentes para proporcionar más información al modelo.
4. **Monitoreo del Modelo:** Implementar un sistema de monitoreo para evaluar el rendimiento del modelo en producción y ajustarlo según sea necesario para mantener su efectividad.

Con estas estrategias, Beta Bank puede mejorar aún más su capacidad para predecir y retener a los clientes, asegurando un enfoque proactivo en la gestión de la satisfacción del cliente y la reducción de la tasa de abandono.
